In [61]:
import pandas as pd
from sklearn import preprocessing
import torch
from torch import optim

from utils import utils
DEVICE = utils.DEVICE

# 0. Prep Data

In [62]:
source_idx = 3
target_idx = 1
winter_idx = 0
summer_idx = 0

train_source_X = pd.read_csv(f"./deep_occupancy_detection/data/{source_idx}_X_train.csv")
target_X = pd.read_csv(f"./deep_occupancy_detection/data/{target_idx}_X_train.csv")
train_source_y_task = pd.read_csv(f"./deep_occupancy_detection/data/{source_idx}_Y_train.csv")[train_source_X.Season==winter_idx].values.reshape(-1)
target_y_task = pd.read_csv(f"./deep_occupancy_detection/data/{target_idx}_Y_train.csv")[target_X.Season==summer_idx].values.reshape(-1)
train_source_X = train_source_X[train_source_X.Season==winter_idx]
target_X = target_X[target_X.Season==summer_idx]

In [63]:
train_source_X

,Energy,Season,Time
1504,67363.495,0,12
1505,237118.680,0,13
1506,78601.300,0,14
1507,274467.298,0,15
1508,492557.058,0,16
...,...,...,...
2075,504342.309,0,39
2076,596620.374,0,40
2077,478971.234,0,41
2078,589670.944,0,42


In [64]:
target_X

,Energy,Season,Time
1248,227903.247,0,12
1249,310118.934,0,13
1250,371554.363,0,14
1251,851215.054,0,15
1252,731550.916,0,16
...,...,...,...
2715,749873.285,0,39
2716,2196655.709,0,40
2717,691079.162,0,41
2718,937151.981,0,42


In [65]:
scaler = preprocessing.StandardScaler()
train_source_X = scaler.fit_transform(train_source_X)
target_X = scaler.fit_transform(target_X)

train_source_X, train_source_y_task = utils.apply_sliding_window(train_source_X, train_source_y_task, filter_len=6)
target_X, target_y_task = utils.apply_sliding_window(target_X, target_y_task, filter_len=6)

In [66]:
source_loader, _, _, _, target_X, target_y_task = utils.get_loader(train_source_X, target_X, train_source_y_task, target_y_task, shuffle=True)

In [67]:
test_thr = int(target_X.shape[0]/2)
test_target_X = target_X[test_thr:]
test_target_y_task = target_y_task[test_thr:]

# 1. p(Y|X) Divergence
## 1.1 Model Fit


In [68]:
feature_extractor = utils.Conv1d(input_size=train_source_X.shape[2]).to(DEVICE)
task_classifier = utils.Decoder(input_size=128, output_size=1).to(DEVICE)
criterion = torch.nn.BCELoss()

feature_optimizer = optim.Adam(feature_extractor.parameters(), lr=0.0001)
task_optimizer = optim.Adam(task_classifier.parameters(), lr=0.0001)

In [69]:
for epoch in range(100):
    for source_data in source_loader:
        source_X, source_y = source_data
        source_y_task = source_y[:, 0]

        source_feature = feature_extractor(source_X)
        source_output = task_classifier(source_feature)
        source_output = torch.sigmoid(source_output).reshape(-1)

        loss = criterion(source_output, source_y_task)

        feature_optimizer.zero_grad()
        task_optimizer.zero_grad()
        loss.backward()
        feature_optimizer.step()
        task_optimizer.step()
    if epoch % 50 == 0:
        print(f"Loss: {loss.item()}")

Loss: 0.6759140491485596
Loss: 0.22195851802825928


## 1.2 Calc Accuracy for Target

In [70]:
pred_y = task_classifier(feature_extractor(target_X))
pred_y = torch.sigmoid(pred_y).reshape(-1)
pred_y = pred_y > 0.5
acc = sum(pred_y == target_y_task) / pred_y.shape[0]
print(f"Accuracy for Target: {acc}")

Accuracy for Target: 0.8241308927536011
